In [ ]:
import os
from pathlib import Path
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# === CONFIGURACIÓN ===
RUTA_CARPETA_ENTRADA = "snapshot_images/snapshot_images"
RUTA_CARPETA_SALIDA = "salida"
EXTENSIONES_VALIDAS = (".png", ".jpg", ".jpeg", ".tif", ".tiff")

PORCENTAJE_CORTE_IZQ = 0.16
PORCENTAJE_CORTE_DER = 0.06
PORCENTAJE_CORTE_SUP = 0.06
PORCENTAJE_CORTE_INF = 0.30

MARGEN_INTERNO_IZQ = 30
MARGEN_INTERNO_DER = 30
MARGEN_INTERNO_SUP = 30
MARGEN_INTERNO_INF = 60

DESVIACION_FONDO = 22
DESVIACION_GAUSS_1 = 2.0
DESVIACION_GAUSS_2 = 4.0
LIMITE_NORMALIZACION = 0.95

PORCENTAJE_SUPERIOR_ESTRICTO = 0.004
TAMANO_KERNEL_NMS_ESTRICTO = 7
RADIO_EXCLUSION_CENTRAL_ESTRICTO = 0.08

PORCENTAJE_SUPERIOR_RELAX = 0.010
TAMANO_KERNEL_NMS_RELAX = 5
RADIO_EXCLUSION_CENTRAL_RELAX = 0.035
MINIMO_PICOS_NECESARIOS = 5

DISTANCIA_MAXIMA_MERGE = 9
MINIMO_PUNTOS_CLUSTER = 1

RADIO_CIRCULO_RESALTADO = 12
GROSOR_CIRCULO = 3
COLOR_CIRCULO = (0, 0, 255)

# === FUNCIONES ===
def recortar_bordes_por_porcentaje(imagen):
    alto, ancho = imagen.shape
    x_inicio = int(round(ancho * PORCENTAJE_CORTE_IZQ))
    x_final = int(round(ancho * (1.0 - PORCENTAJE_CORTE_DER)))
    y_inicio = int(round(alto * PORCENTAJE_CORTE_SUP))
    y_final = int(round(alto * (1.0 - PORCENTAJE_CORTE_INF)))
    x_inicio = max(0, min(x_inicio, ancho - 2))
    x_final = max(x_inicio + 1, min(x_final, ancho))
    y_inicio = max(0, min(y_inicio, alto - 2))
    y_final = max(y_inicio + 1, min(y_final, alto))
    return imagen[y_inicio:y_final, x_inicio:x_final], x_inicio, y_inicio

def calcular_mapa_diferencia_gauss(imagen_gris):
    imagen_float = imagen_gris.astype(np.float32) / 255.0
    fondo_suavizado = cv2.GaussianBlur(imagen_float, (0, 0), DESVIACION_FONDO)
    imagen_base = np.clip(imagen_float - fondo_suavizado, 0, 1)
    imagen_base = np.minimum(imagen_base, LIMITE_NORMALIZACION)
    gauss1 = cv2.GaussianBlur(imagen_base, (0, 0), DESVIACION_GAUSS_1)
    gauss2 = cv2.GaussianBlur(imagen_base, (0, 0), DESVIACION_GAUSS_2)
    diferencia_gauss = gauss1 - gauss2
    diferencia_gauss = np.clip(diferencia_gauss, 0, None)
    maximo_valor = diferencia_gauss.max()
    if maximo_valor > 0:
        diferencia_gauss /= maximo_valor
    return diferencia_gauss

def detectar_picos_locales(mapa_intensidad, porcentaje_superior, tamano_kernel):
    valores_planos = mapa_intensidad.ravel()
    k = max(1, int(round((1.0 - porcentaje_superior) * valores_planos.size)))
    umbral = np.partition(valores_planos, k)[k]
    candidatos = mapa_intensidad >= umbral
    tamano_kernel = int(tamano_kernel) | 1
    maximos_locales = cv2.dilate(
        mapa_intensidad, np.ones((tamano_kernel, tamano_kernel), np.uint8)
    )
    picos = candidatos & (mapa_intensidad >= maximos_locales - 1e-12)
    coordenadas_y, coordenadas_x = np.where(picos)
    valores_picos = mapa_intensidad[coordenadas_y, coordenadas_x]
    return coordenadas_x, coordenadas_y, valores_picos

def excluir_pico_central(mapa_intensidad, coordenadas_x, coordenadas_y, radio_relativo):
    alto, ancho = mapa_intensidad.shape
    centro_y, centro_x = np.unravel_index(
        np.argmax(mapa_intensidad), mapa_intensidad.shape
    )
    radio_exclusion = int(max(10, radio_relativo * max(alto, ancho)))
    mantener = (
        (coordenadas_x - centro_x) ** 2 + (coordenadas_y - centro_y) ** 2
    ) > radio_exclusion**2
    return coordenadas_x[mantener], coordenadas_y[mantener]

def excluir_margen_interno(coordenadas_x, coordenadas_y, forma_imagen):
    alto, ancho = forma_imagen
    mantener = (
        (coordenadas_x >= MARGEN_INTERNO_IZQ)
        & (coordenadas_x < ancho - MARGEN_INTERNO_DER)
        & (coordenadas_y >= MARGEN_INTERNO_SUP)
        & (coordenadas_y < alto - MARGEN_INTERNO_INF)
    )
    return coordenadas_x[mantener], coordenadas_y[mantener]

def fusionar_puntos_dbscan(coordenadas_x, coordenadas_y):
    if coordenadas_x.size == 0:
        return []
    datos = np.stack([coordenadas_x, coordenadas_y], axis=1).astype(np.float32)
    etiquetas = DBSCAN(
        eps=DISTANCIA_MAXIMA_MERGE, min_samples=MINIMO_PUNTOS_CLUSTER
    ).fit_predict(datos)
    puntos_fusionados = []
    for etiqueta in sorted(set(etiquetas)):
        if etiqueta == -1:
            continue
        mascara = etiquetas == etiqueta
        puntos_fusionados.append(
            (
                int(round(coordenadas_x[mascara].mean())),
                int(round(coordenadas_y[mascara].mean())),
            )
        )
    return puntos_fusionados

def detectar_puntos_en_panel(panel, porcentaje_top, kernel_nms, radio_exclusion):
    mapa_picos = calcular_mapa_diferencia_gauss(panel)
    coords_x, coords_y, _ = detectar_picos_locales(
        mapa_picos, porcentaje_top, kernel_nms
    )
    coords_x, coords_y = excluir_pico_central(
        mapa_picos, coords_x, coords_y, radio_exclusion
    )
    coords_x, coords_y = excluir_margen_interno(coords_x, coords_y, panel.shape)
    return fusionar_puntos_dbscan(coords_x, coords_y)

def procesar_imagen(ruta_imagen):
    imagen_gris = cv2.imread(str(ruta_imagen), cv2.IMREAD_GRAYSCALE)
    if imagen_gris is None:
        raise FileNotFoundError(ruta_imagen)
    panel_recortado, offset_x, offset_y = recortar_bordes_por_porcentaje(imagen_gris)
    centros_detectados = detectar_puntos_en_panel(
        panel_recortado,
        PORCENTAJE_SUPERIOR_ESTRICTO,
        TAMANO_KERNEL_NMS_ESTRICTO,
        RADIO_EXCLUSION_CENTRAL_ESTRICTO,
    )
    if len(centros_detectados) < MINIMO_PICOS_NECESARIOS:
        puntos_extra = detectar_puntos_en_panel(
            panel_recortado,
            PORCENTAJE_SUPERIOR_RELAX,
            TAMANO_KERNEL_NMS_RELAX,
            RADIO_EXCLUSION_CENTRAL_RELAX,
        )
        todos_los_puntos = centros_detectados + puntos_extra
        if todos_los_puntos:
            x_todos = np.array([c[0] for c in todos_los_puntos], dtype=np.int32)
            y_todos = np.array([c[1] for c in todos_los_puntos], dtype=np.int32)
            centros_detectados = fusionar_puntos_dbscan(x_todos, y_todos)
        else:
            centros_detectados = []
    centros_absolutos = [(x + offset_x, y + offset_y) for (x, y) in centros_detectados]
    imagen_color = cv2.cvtColor(imagen_gris, cv2.COLOR_GRAY2BGR)
    for x, y in centros_absolutos:
        cv2.circle(
            imagen_color,
            (x, y),
            RADIO_CIRCULO_RESALTADO,
            COLOR_CIRCULO,
            GROSOR_CIRCULO,
            lineType=cv2.LINE_AA,
        )
    return imagen_color

# === PROCESAMIENTO Y VISUALIZACIÓN ===
carpeta_entrada = Path(RUTA_CARPETA_ENTRADA)
carpeta_salida = Path(RUTA_CARPETA_SALIDA)
carpeta_salida.mkdir(parents=True, exist_ok=True)

imagenes = [p for p in carpeta_entrada.iterdir() if p.suffix.lower() in EXTENSIONES_VALIDAS]
print(f"Procesando {len(imagenes)} imágenes…")

for ruta_imagen in imagenes:
    try:
        imagen_procesada = procesar_imagen(ruta_imagen)
        cv2.imwrite(str(carpeta_salida / ruta_imagen.name), imagen_procesada)

        # Mostrar dentro del notebook 
        plt.figure(figsize=(8, 6))
        plt.imshow(cv2.cvtColor(imagen_procesada, cv2.COLOR_BGR2RGB))
        plt.title(f"Resultado: {ruta_imagen.name}")
        plt.axis('off')
        plt.show()

    except Exception as error:
        print(f"[ERROR] {ruta_imagen.name}: {error}")

print(f"\n Proceso completado. Resultados guardados en: {RUTA_CARPETA_SALIDA}")
